# [Advent of Code 2020 Day 9](https://adventofcode.com/2020/day/9)

This looks interesting. Seems like 2SUM's cousin mixed with sliding window? Maybe rolling hash? jk

## Initial setup

In [299]:
import ipytest
import sys
sys.path.append("..")
from ansi import *
from comp import *
ipytest.autoconfig()

## 2SUM
Brute force version of the decision problem version of 2SUM: determine if there exist two numbers in the inclusive `[l, r]` range that add up to target. Needed for the brute force implementation.

In [300]:
def two_sum_decision_brute_force(nums: list[int], l: int, r: int, target: int) -> bool:
    for i in range(l, r + 1):
        for j in range(i + 1, r + 1):
            if nums[i] + nums[j] == target:
                return True
    return False

In [301]:
%%ipytest
def test_two_sum_decision_brute_force():
    assert two_sum_decision_brute_force([4, 5, 6, 7], 0, 3, 11) is True
    assert two_sum_decision_brute_force([4, 5, 6, 7], 0, 1, 11) is False
    assert two_sum_decision_brute_force([4, 5, 6, 7], 0, 1, 9) is True
    assert two_sum_decision_brute_force([4, 5, 6, 7], 2, 3, 13) is True
    assert two_sum_decision_brute_force([1, 2, 3, 4], 0, 3, 70) is False

.                                                                                            [100%]
1 passed in 0.01s


## Brute Force Implementation
Exhaustively examine every pair within the sliding window.

In [302]:
def find_error_brute_force(nums: list[int], preamble_size: int, window_size: int) -> int | None:
    """
    Given an array of integers, determines the first integer that does not respect the 2SUM
    property of its preceding sliding window.
    :param nums: the integers to examine
    :param preamble_size: the number of pre-set values in the array (ergo where to ignore)
    :param window_size: the number of integers to look back and consider
    :return: the index of the nums array representing the violating integer
    """

    assert preamble_size - window_size >= 0

    for idx in range(preamble_size, len(nums)):
        if two_sum_decision_brute_force(nums, idx - window_size, idx - 1, nums[idx]) is False:
            return idx

    return None

In [303]:
%%ipytest
def test_find_error_brute_force():
    assert find_error_brute_force([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 5, 5) == 5
    assert find_error_brute_force([1, 2, 3, 5, 8, 13, 21, 4], 3, 3) == 7
    assert find_error_brute_force([35, 20, 15, 25, 47, 40, 62, 55, 65, 95, 102, 117, 150, 182, 127, 219, 299, 277, 309, 576], 5, 5) == 14

.                                                                                            [100%]
1 passed in 0.01s


Nice, that worked, and it passes part 1. It's not the fastest, but it works. Part 2 will likely involve speed though.

## Part 2: Exact Subarray Sum
Given a list of numbers, find a subarray from `[i..j]` that adds up to `target`, then return `i` + `j`.

### Naive Get Sum
This will just add up the integers on the range `[i..j]` inclusive. $O(n)$.

In [304]:
def get_sum_on_inclusive_range(nums, l, r):
    the_sum = 0
    for i in range(l, r + 1):
        the_sum += nums[i]
    return the_sum

In [305]:
%%ipytest
def test_get_sum_on_inclusive_range():
    assert get_sum_on_inclusive_range([1, 2, 3, 4], 1, 2) == 5
    assert get_sum_on_inclusive_range([1, 2, 3, 4], 0, 0) == 1
    assert get_sum_on_inclusive_range([1, 2, 3, 4], 3, 3) == 4
    assert get_sum_on_inclusive_range(list(range(1, 100 + 1)), 0, 99) == 5050

.                                                                                            [100%]
1 passed in 0.01s


In [306]:
def exact_subarray_sum(nums: list[int], target: int) -> int:
    """
    Return the sum of the min and max of the first range adding up to target.
    :param nums: numbers to examine
    :param target: number to add up to
    :return: the sum of the min and max representing a subarray satisfying this
    """
    for i in range(len(nums)):
        for j in range(i + 1, len(nums)):
            if get_sum_on_inclusive_range(nums, i, j) == target:
                arr_slice = nums[i : j + 1]
                return min(arr_slice) + max(arr_slice)
    raise Exception("Should have found something... poorly formed arguments maybe?")

In [307]:
%%ipytest
def test_exact_subarray_sum():
    assert exact_subarray_sum([35, 20, 15, 25, 47, 40, 62, 55, 65, 95, 102, 117, 150, 182, 127, 219, 299, 277, 309, 576], 127) == 62

.                                                                                            [100%]
1 passed in 0.01s


And that solves part 2. It's pretty damn slow!

## Main Solver

In [308]:
def solve(prob, filename):
    lines = []
    gen = yield_line(filename)

    for line in gen:
        lines.append(int(line))

    nums = lines

    answer_to_part_1 = nums[find_error_brute_force(nums, 25, 25)]

    if prob == 1:
        return answer_to_part_1
    elif prob == 2:
        return exact_subarray_sum(nums, answer_to_part_1)
    else:
        print("Invalid problem code")
        exit()

In [309]:
%%ipytest
def test_solve():
    assert solve(1, "example1") == 50
#    assert solve(1, "example2") == 127
    assert solve(1, "input") == 1504371145
#    assert solve(2, "example2") == 62
    assert solve(2, "input") == 183278487

.                                                                                            [100%]
1 passed in 5.65s
